# Zero-Sum Infinite Sets Theory (TNSIM) Demonstration

This notebook demonstrates the core concepts and practical applications of TNSIM.

## Contents
1. Core Concepts
2. Mathematical Foundations
3. Practical Examples
4. Compensation Algorithms
5. Convergence Analysis
6. Balansis Integration
7. Zero-Sum Attention Mechanism

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Any

# Import TNSIM components
import sys
sys.path.append('../')

from tnsim.core.zero_sum_set import ZeroSumInfiniteSet
from tnsim.core.cache import TNSIMCache
from tnsim.core.parallel import ParallelTNSIM
from tnsim.integrations.balansis_integration import BalansisCompensator, ZeroSumAttention

# Visualization setup
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

print('Libraries successfully imported!')

## 1. TNSIM Core Concepts

Zero-Sum Infinite Sets Theory is based on the compensation principle:

**Definition**: A set S is called a zero-sum set if:
```
∑(s ∈ S) s = 0
```

**Compensation operation ⊕**: For any elements a, b:
```
a ⊕ b = compensated_sum(a, b) with minimal rounding error


In [ ]:
# Demonstration of core concepts

def demonstrate_basic_concepts():
    """Demonstration of TNSIM core concepts."""
    
    print('=== TNSIM Core Concepts ===\n')
    
    # 1. Creating a simple zero-sum set
    elements = [1.0, -2.0, 0.5, 0.5]
    zero_sum_set = ZeroSumInfiniteSet(elements)
    
    print(f'Set elements: {elements}')
    print(f'Sum of elements: {sum(elements)}')
    print(f'Is zero-sum set: {zero_sum_set.validate_zero_sum()["is_valid"]}\n')
    
    # 2. Demonstration of floating-point precision issues
    problematic_elements = [0.1, 0.2, -0.3]
    naive_sum = sum(problematic_elements)
    
    problematic_set = ZeroSumInfiniteSet(problematic_elements)
    compensated_result = problematic_set.zero_sum_operation(method='compensated')
    
    print(f'Problematic elements: {problematic_elements}')
    print(f'Naive sum: {naive_sum:.17f}')
    print(f'Compensated sum: {compensated_result:.17f}')
    print(f'Precision improvement: {abs(naive_sum) / abs(compensated_result):.2f}x\n')
    
    return zero_sum_set, problematic_set

basic_demo_results = demonstrate_basic_concepts()

## 2. Mathematical Foundations

### Kahan Algorithm for Compensated Summation

The Kahan algorithm minimizes rounding errors during summation:

```python
def kahan_sum(numbers):
    sum_val = 0.0
    compensation = 0.0
    
    for num in numbers:
        y = num - compensation
        t = sum_val + y
        compensation = (t - sum_val) - y
        sum_val = t
    
    return sum_val
```

In [ ]:
# Demonstration of mathematical foundations

def demonstrate_mathematical_foundations():
    """Demonstration of TNSIM mathematical foundations."""
    
    print('=== Mathematical Foundations ===\n')
    
    # Creating test data with potential rounding errors
    n = 10000
    small_numbers = [1e-10] * n + [-1e-10 * n]
    
    # Comparison of different summation methods
    naive_sum = sum(small_numbers)
    
    test_set = ZeroSumInfiniteSet(small_numbers)
    compensated_sum = test_set.zero_sum_operation(method='compensated')
    stabilized_sum = test_set.zero_sum_operation(method='stabilized')
    
    print(f'Testing with {n} small numbers:')
    print(f'Naive summation: {naive_sum:.2e}')
    print(f'Compensated summation: {compensated_sum:.2e}')
    print(f'Stabilized summation: {stabilized_sum:.2e}\n')
    
    # Error analysis
    theoretical_sum = 0.0  # Theoretically should be 0
    
    naive_error = abs(naive_sum - theoretical_sum)
    compensated_error = abs(compensated_sum - theoretical_sum)
    stabilized_error = abs(stabilized_sum - theoretical_sum)
    
    print('Error analysis:')
    print(f'Naive method error: {naive_error:.2e}')
    print(f'Compensated method error: {compensated_error:.2e}')
    print(f'Stabilized method error: {stabilized_error:.2e}\n')
    
    if compensated_error > 0:
        improvement_factor = naive_error / compensated_error
        print(f'Accuracy improvement: {improvement_factor:.2f}x')
    
    return {
        'naive_sum': naive_sum,
        'compensated_sum': compensated_sum,
        'stabilized_sum': stabilized_sum,
        'errors': {
            'naive': naive_error,
            'compensated': compensated_error,
            'stabilized': stabilized_error
        }
    }

math_demo_results = demonstrate_mathematical_foundations()

## 3. Practical Examples

### Applications in various fields:

1. **Financial calculations**: Precise transaction summation
2. **Scientific computing**: Error compensation in numerical methods
3. **Machine learning**: Gradient stabilization
4. **Signal processing**: Noise minimization during summation

In [ ]:
# Practical application examples

def demonstrate_practical_examples():
    """Demonstration of practical TNSIM application examples."""
    
    print('=== Practical Examples ===\n')
    
    # Example 1: Financial calculations
    print('1. Financial calculations:')
    transactions = [1000.01, -999.99, 0.50, -0.52, 100.00, -100.00]
    
    financial_set = ZeroSumInfiniteSet(transactions)
    balance = financial_set.zero_sum_operation(method='compensated')
    validation = financial_set.validate_zero_sum(tolerance=1e-10)
    
    print(f'  Transactions: {transactions}')
    print(f'  Final balance: {balance:.10f}')
    print(f'  Validation: {validation["is_valid"]} (error: {validation["error"]:.2e})\n')
    
    # Example 2: Rounding error compensation
    print('2. Rounding error compensation:')
    
    # Create a situation with error accumulation
    base_value = 0.1
    iterations = 1000
    
    # Naive approach
    naive_result = 0.0
    for _ in range(iterations):
        naive_result += base_value
    naive_result -= iterations * base_value
    
    # Compensation approach
    compensation_elements = [base_value] * iterations + [-base_value * iterations]
    compensation_set = ZeroSumInfiniteSet(compensation_elements)
    compensated_result = compensation_set.zero_sum_operation(method='compensated')
    
    print(f'  Naive result: {naive_result:.15f}')
    print(f'  Compensated result: {compensated_result:.15f}')
    print(f'  Improvement: {abs(naive_result) / abs(compensated_result):.2f}x\n')
    
    # Example 3: Gradient stabilization
    print('3. Gradient stabilization (simulation):')
    
    # Simulate gradients with noise
    np.random.seed(42)
    gradients = []
    for i in range(100):
        gradient = np.sin(i * 0.1) + np.random.normal(0, 0.01)
        gradients.append(gradient)
    
    # Add compensating gradient
    gradients.append(-sum(gradients))
    
    gradient_set = ZeroSumInfiniteSet(gradients)
    stabilized_gradient = gradient_set.zero_sum_operation(method='stabilized')
    
    print(f'  Number of gradients: {len(gradients)}')
    print(f'  Stabilized sum: {stabilized_gradient:.10f}')
    print(f'  Standard deviation: {np.std(gradients[:-1]):.6f}')
    
    return {
        'financial_balance': balance,
        'compensation_improvement': abs(naive_result) / abs(compensated_result) if compensated_result != 0 else float('inf'),
        'gradient_stabilization': stabilized_gradient
    }

practical_demo_results = demonstrate_practical_examples()

## 4. Compensation Algorithms

TNSIM provides several compensation algorithms:

- **Direct**: Direct summation
- **Compensated**: Kahan algorithm
- **Iterative**: Iterative compensation
- **Adaptive**: Adaptive compensation
- **Stabilized**: Stabilized summation

In [ ]:
# Comparison of compensation algorithms

def compare_compensation_algorithms():
    """Comparison of different compensation algorithms."""
    
    print('=== Compensation Algorithm Comparison ===\n')
    
    # Create test set with potential errors
    np.random.seed(123)
    n = 5000
    elements = np.random.uniform(-1, 1, n).tolist()
    elements.append(-sum(elements))  # Make sum equal to zero
    
    test_set = ZeroSumInfiniteSet(elements)
    
    # Test different methods
    methods = ['direct', 'compensated', 'iterative', 'adaptive', 'stabilized']
    results = {}
    
    print(f'Testing with {len(elements)} elements:\n')
    
    for method in methods:
        try:
            import time
            start_time = time.time()
            
            result = test_set.zero_sum_operation(method=method)
            validation = test_set.validate_zero_sum(tolerance=1e-12)
            
            end_time = time.time()
            execution_time = end_time - start_time
            
            results[method] = {
                'result': result,
                'error': validation['error'],
                'is_valid': validation['is_valid'],
                'time': execution_time
            }
            
            print(f'{method.capitalize():>12}: {result:.2e} | Error: {validation["error"]:.2e} | Time: {execution_time*1000:.2f}ms')
            
        except Exception as e:
            print(f'{method.capitalize():>12}: ERROR - {str(e)}')
            results[method] = {'error': str(e)}
    
    print('\n' + '='*60)
    
    # Results analysis
    valid_results = {k: v for k, v in results.items() if 'result' in v}
    
    if valid_results:
        best_accuracy = min(valid_results.values(), key=lambda x: x['error'])
        fastest = min(valid_results.values(), key=lambda x: x['time'])
        
        best_accuracy_method = [k for k, v in valid_results.items() if v['error'] == best_accuracy['error']][0]
        fastest_method = [k for k, v in valid_results.items() if v['time'] == fastest['time']][0]
        
        print(f'Most accurate method: {best_accuracy_method} (error: {best_accuracy["error"]:.2e})')
        print(f'Fastest method: {fastest_method} (time: {fastest["time"]*1000:.2f}ms)')
    
    return results

compensation_comparison = compare_compensation_algorithms()

## 5. Convergence Analysis

TNSIM provides tools for analyzing series convergence:

- **Ratio Test**: Convergence check using D'Alembert's criterion
- **Root Test**: Convergence check using Cauchy's criterion
- **Integral Test**: Comparison with integral
- **Partial Sums Analysis**: Investigation of partial sums behavior

In [ ]:
# Convergence analysis demonstration

def demonstrate_convergence_analysis():
    """Demonstration of convergence analysis for various series."""
    
    print('=== Convergence Analysis ===\n')
    
    # Test different types of series
    series_types = ['harmonic', 'alternating', 'geometric']
    
    for series_type in series_types:
        print(f'Analysis of {series_type} series:')
        
        # Create series
        if series_type == 'harmonic':
            elements = [1/i for i in range(1, 101)]
        elif series_type == 'alternating':
            elements = [(-1)**i / i for i in range(1, 101)]
        elif series_type == 'geometric':
            r = 0.5
            elements = [r**i for i in range(100)]
        
        # Add compensating element to create zero-sum set
        elements.append(-sum(elements))
        
        series_set = ZeroSumInfiniteSet(elements, series_type=series_type)
        
        # Convergence analysis
        convergence = series_set.convergence_analysis()
        
        print(f'  Converges: {convergence["converges"]}')
        print(f'  Convergence type: {convergence["convergence_type"]}')
        print(f'  Convergence rate: {convergence["convergence_rate"]:.6f}')
        
        if 'partial_sums' in convergence:
            partial_sums = convergence['partial_sums']
            print(f'  Last partial sums: {partial_sums[-3:]}')
        
        print()
    
    # Convergence visualization (for geometric series)
    print('Geometric series convergence visualization:')
    
    r = 0.7
    n_terms = 50
    geometric_elements = [r**i for i in range(n_terms)]
    
    # Calculate partial sums
    partial_sums = []
    current_sum = 0
    for element in geometric_elements:
        current_sum += element
        partial_sums.append(current_sum)
    
    # Theoretical limit for geometric series
    theoretical_limit = 1 / (1 - r)
    
    print(f'  Theoretical limit: {theoretical_limit:.6f}')
    print(f'  Partial sum ({n_terms} elements): {partial_sums[-1]:.6f}')
    print(f'  Error: {abs(theoretical_limit - partial_sums[-1]):.6f}')
    
    return {
        'geometric_partial_sums': partial_sums,
        'theoretical_limit': theoretical_limit,
        'convergence_error': abs(theoretical_limit - partial_sums[-1])
    }

convergence_demo = demonstrate_convergence_analysis()

## 6. Balansis Integration

Balansis provides additional compensation capabilities:

- **BalansisCompensator**: High-precision compensation with quality metrics
- **Adaptive algorithms**: Automatic selection of optimal method
- **Quality analysis**: Assessment of compensation effectiveness

In [ ]:
# Balansis integration demonstration

def demonstrate_balansis_integration():
    """Demonstration of integration with Balansis library."""
    
    print('=== Balansis Integration ===\n')
    
    # Create Balansis compensator
    compensator = BalansisCompensator(precision='high')
    
    # Test 1: Series compensation
    print('1. Harmonic series compensation:')
    harmonic_series = [1/i for i in range(1, 1001)]
    
    balansis_result, metrics = compensator.compensate_series(harmonic_series)
    
    print(f'  Compensation result: {balansis_result:.10f}')
    print(f'  Compensation quality: {metrics.quality_score:.6f}')
    print(f'  Compensation error: {metrics.compensation_error:.2e}')
    print(f'  Execution time: {metrics.execution_time*1000:.2f}ms\n')
    
    # Test 2: Zero-sum set compensation
    print('2. Zero-sum set compensation:')
    
    # Create complex set
    np.random.seed(456)
    complex_elements = []
    for i in range(500):
        element = np.sin(i/50) * np.exp(-i/100) + np.random.normal(0, 0.0001)
        complex_elements.append(element)
    
    complex_elements.append(-sum(complex_elements))
    complex_set = ZeroSumInfiniteSet(complex_elements, series_type='custom')
    
    # Compare standard and Balansis compensation
    standard_result = complex_set.zero_sum_operation(method='compensated')
    balansis_set_result, balansis_metrics = compensator.compensate_zero_sum_set(complex_set)
    
    print(f'  Standard compensation: {standard_result:.12f}')
    print(f'  Balansis compensation: {balansis_set_result:.12f}')
    print(f'  Accuracy improvement: {abs(standard_result) / abs(balansis_set_result):.2f}x')
    print(f'  Balansis quality: {balansis_metrics.quality_score:.6f}\n')
    
    # Test 3: Analysis of different precision levels
    print('3. Precision levels comparison:')
    
    precision_levels = ['standard', 'high', 'ultra']
    test_series = [0.1, 0.01, 0.001] * 100 + [-sum([0.1, 0.01, 0.001] * 100)]
    
    for precision in precision_levels:
        try:
            prec_compensator = BalansisCompensator(precision=precision)
            result, metrics = prec_compensator.compensate_series(test_series)
            
            print(f'  {precision.capitalize():>8}: {result:.2e} | Quality: {metrics.quality_score:.4f}')
        except Exception as e:
            print(f'  {precision.capitalize():>8}: ERROR - {str(e)}')
    
    return {
        'harmonic_compensation': balansis_result,
        'harmonic_metrics': metrics,
        'complex_set_improvement': abs(standard_result) / abs(balansis_set_result) if balansis_set_result != 0 else float('inf'),
        'balansis_quality': balansis_metrics.quality_score
    }

balansis_demo = demonstrate_balansis_integration()

## 7. Zero-Sum Attention Mechanism

Zero-Sum Attention is an innovative attention mechanism based on TNSIM principles:

- **Compensated attention weights**: Minimization of weight errors
- **Gradient stabilization**: Improved neural network training
- **Energy conservation**: Maintaining balance in activations

In [ ]:
# Zero-Sum Attention demonstration

def demonstrate_zero_sum_attention():
    """Demonstration of Zero-Sum Attention mechanism."""
    
    print('=== Zero-Sum Attention ===\n')
    
    # Create Zero-Sum Attention model
    d_model = 64
    n_heads = 8
    seq_length = 32
    batch_size = 4
    
    attention = ZeroSumAttention(
        d_model=d_model,
        n_heads=n_heads,
        compensation_strength=0.1
    )
    
    print(f'Created Zero-Sum Attention model:')
    print(f'  Model dimension: {d_model}')
    print(f'  Number of heads: {n_heads}')
    print(f'  Sequence length: {seq_length}')
    print(f'  Batch size: {batch_size}\n')
    
    # Create test data
    np.random.seed(789)
    x = np.random.randn(batch_size, seq_length, d_model).astype(np.float32)
    
    # Forward pass
    print('Performing forward pass...')
    
    try:
        import time
        start_time = time.time()
        
        output, attention_weights = attention.forward(x)
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        print(f'  Execution time: {execution_time*1000:.2f}ms')
        print(f'  Output shape: {output.shape}')
        print(f'  Attention weights shape: {attention_weights.shape}\n')
        
        # Attention weights analysis
        print('Attention weights analysis:')
        
        # Check weight sums along last dimension (should be ~1)
        weight_sums = np.sum(attention_weights, axis=-1)
        mean_sum = np.mean(weight_sums)
        std_sum = np.std(weight_sums)
        
        print(f'  Average weight sum: {mean_sum:.6f}')
        print(f'  Standard deviation of sums: {std_sum:.6f}')
        
        # Weight entropy analysis
        def calculate_entropy(weights):
            # Avoid log(0)
            weights = np.clip(weights, 1e-10, 1.0)
            return -np.sum(weights * np.log(weights), axis=-1)
        
        entropies = calculate_entropy(attention_weights)
        mean_entropy = np.mean(entropies)
        
        print(f'  Average weight entropy: {mean_entropy:.4f}')
        print(f'  Maximum entropy: {np.log(seq_length):.4f}\n')
        
        # Stability check
        print('Stability check:')
        
        output_mean = np.mean(output)
        output_std = np.std(output)
        
        print(f'  Output mean: {output_mean:.6f}')
        print(f'  Output standard deviation: {output_std:.6f}')
        
        # Check for NaN or Inf
        has_nan = np.isnan(output).any()
        has_inf = np.isinf(output).any()
        
        print(f'  Contains NaN: {has_nan}')
        print(f'  Contains Inf: {has_inf}')
        
        return {
            'execution_time': execution_time,
            'output_shape': output.shape,
            'attention_weights_shape': attention_weights.shape,
            'mean_weight_sum': mean_sum,
            'weight_sum_std': std_sum,
            'mean_entropy': mean_entropy,
            'output_stats': {
                'mean': output_mean,
                'std': output_std,
                'has_nan': has_nan,
                'has_inf': has_inf
            }
        }
        
    except Exception as e:
        print(f'ERROR executing Zero-Sum Attention: {str(e)}')
        return {'error': str(e)}

attention_demo = demonstrate_zero_sum_attention()

In [ ]:
# Final demonstration: comprehensive example

def comprehensive_tnsim_demo():
    """Comprehensive demonstration of TNSIM capabilities."""
    
    print('Comprehensive TNSIM demonstration:')
    print('=' * 50)
    
    # 1. Create complex set
    complex_elements = []
    for i in range(1, 101):
        element = np.sin(i/10) * np.exp(-i/50) + np.random.normal(0, 0.001)
        complex_elements.append(element)
    
    # Add compensating element
    complex_elements.append(-sum(complex_elements))
    
    complex_set = ZeroSumInfiniteSet(complex_elements, series_type='custom')
    
    print(f'Created set with {len(complex_elements)} elements')
    print(f'Original sum: {sum(complex_elements[:-1]):.10f}')
    print(f'Compensating element: {complex_elements[-1]:.10f}')
    
    # 2. Apply different methods
    methods = ['direct', 'iterative', 'adaptive']
    results = {}
    
    for method in methods:
        result = complex_set.zero_sum_operation(method=method)
        validation = complex_set.validate_zero_sum(tolerance=1e-12)
        
        results[method] = {
            'result': result,
            'error': validation['error'],
            'is_valid': validation['is_valid']
        }
        
        print(f'\nMethod {method}:')
        print(f'  Result: {result:.12f}')
        print(f'  Error: {validation["error"]:.2e}')
        print(f'  Valid: {validation["is_valid"]}')
    
    # 3. Convergence analysis
    convergence = complex_set.convergence_analysis()
    print(f'\nConvergence analysis:')
    print(f'  Converges: {convergence["converges"]}')
    print(f'  Type: {convergence["convergence_type"]}')
    print(f'  Rate: {convergence["convergence_rate"]:.8f}')
    
    # 4. Balansis integration
    compensator = BalansisCompensator()
    balansis_result, metrics = compensator.compensate_zero_sum_set(complex_set)
    
    print(f'\nBalansis integration:')
    print(f'  Result: {balansis_result:.12f}')
    print(f'  Quality: {metrics.quality_score:.6f}')
    print(f'  Compensation error: {metrics.compensation_error:.2e}')
    
    return {
        'complex_set': complex_set,
        'method_results': results,
        'convergence': convergence,
        'balansis_result': balansis_result,
        'balansis_metrics': metrics
    }

# Execute comprehensive demonstration
final_demo = comprehensive_tnsim_demo()

print('\n' + '='*60)
print('DEMONSTRATION COMPLETED SUCCESSFULLY!')
print('='*60)
print('\nZero-Sum Infinite Sets Theory provides')
print('powerful tools for high-precision calculations and')
print('error compensation in various application domains.')
print('\nFor additional information, please refer to')
print('TNSIM documentation and examples in the repository.')